In [ ]:
# Current directory
import os
os.chdir('E:\work\mine')

### 导入数据

In [ ]:
import numpy as np
import pandas as pd

X_new=pd.read_csv(r"extract\All_feature\mRMR\TPC_Group1\mRMR_TAPA_500.csv", header=None)
y_new= pd.read_csv(r'extract\label.csv', header=None)

# y_new = y_new.values.ravel()
print(X_new.shape)
print(y_new.shape)
X_new = np.array(X_new)
y_new = np.array(y_new).ravel()

### 数据标准化

In [ ]:
from sklearn.preprocessing import StandardScaler

# 使用 StandardScaler 进行标准化
scaler = StandardScaler()
X_new = scaler.fit_transform(X_new)

### 数据集划分

In [ ]:
# dataset splitting
from sklearn.model_selection import train_test_split
X_train_whole, X_ind_test, y_train_whole, y_ind_test =  train_test_split( X_new, y_new, test_size=0.2, random_state=1111)

print(X_train_whole.shape)
print(X_ind_test.shape)

### 10折交叉验证

In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, roc_auc_score
import numpy as np

X_train_whole, X_ind_test, y_train_whole, y_ind_test =  train_test_split( X_new, y_new, test_size=0.2, random_state=1111)
BACC_collection = []
ACC_collection = []
Sn_collection = []
Sp_collection = []
MCC_collection = []
AUC_collection = []

def categorical_probas_to_classes(p):
    return np.argmax(p, axis=1)

svm_model = SVC(C = 1,degree=1, kernel='rbf',  probability=True, random_state=1111)
lr_model = LogisticRegression(C = 0.014174741629268055 ,max_iter=5000)
pls_model = PLSRegression(max_iter= 100, n_components=3, scale=True, tol=0.0001)

skf = StratifiedKFold(n_splits=10)
for train, test in skf.split(X_train_whole, y_train_whole):
    X_train, X_valid, y_train, y_valid = X_train_whole[train], X_train_whole[test], y_train_whole[train], y_train_whole[test]

    # 训练模型
    svm_model.fit(X_train, y_train)
    lr_model.fit(X_train, y_train)
    pls_model.fit(X_train, y_train)


    # 预测
    svm_proba = svm_model.predict_proba(X_valid) 
    svm_proba = svm_proba[:, 1]

    lr_proba = lr_model.predict_proba(X_valid) 
    lr_proba = lr_proba[:, 1]

    pls_proba = pls_model.predict(X_valid) 
    pls_proba = np.clip(pls_proba, 0, 1)
    pls_proba = np.reshape(pls_proba, -1)   # 将二维数组展平为一维

    # 平均投票
    y_pred_proba = (lr_proba  + svm_proba +pls_proba + et_proba + nb_proba + knn_proba)/6
    y_valid_pred = np.where(y_pred_proba < 0.5, 0, 1)

    TP, FP, FN, TN = confusion_matrix(y_valid, y_valid_pred).ravel()
    Sn_collection.append(TP / (TP + FN))
    Sp_collection.append(TN / (TN + FP))
    MCC = (TP*TN-FP*FN)/np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
    MCC_collection.append(MCC)
    ACC_collection.append((TP + TN) / (TP + TN + FP + FN))
    BACC_collection.append(0.5 * (TP / (TP + FN)) + 0.5 * (TN / (TN + FP)))
    auc = roc_auc_score(y_valid, y_pred_proba)
    AUC_collection.append(auc)

# 输出结果
print(round(np.mean(ACC_collection),3), '±', round(np.std(ACC_collection),3))
print(round(np.mean(Sn_collection),3), '±', round(np.std(Sn_collection),3))
print(round(np.mean(Sp_collection),3), '±', round(np.std(Sp_collection),3))
print(round(np.mean(MCC_collection),3), '±', round(np.std(MCC_collection),3))
print(round(np.mean(AUC_collection),3), '±', round(np.std(AUC_collection),3))



In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, roc_auc_score
import numpy as np
import statistics
from sklearn.metrics import confusion_matrix
import math
from sklearn.metrics import roc_auc_score

X_train_whole, X_ind_test, y_train_whole, y_ind_test =  train_test_split( X_new, y_new, test_size=0.2, random_state=1111)
BACC_collection = []
ACC_collection = []
Sn_collection = []
Sp_collection = []
MCC_collection = []
AUC_collection = []

def categorical_probas_to_classes(p):
    return np.argmax(p, axis=1)
for i in range(10):
    # dataset splitting
    X_train_whole, X_ind_test, y_train_whole, y_ind_test = train_test_split(X_new, y_new, test_size=0.2, random_state=i)
    svm_model = SVC(C = 1,degree=1, kernel='rbf',  probability=True, random_state=1111)
    lr_model = LogisticRegression(C =  0.014174741629268055 ,max_iter=5000)
    pls_model = PLSRegression(max_iter= 100, n_components=3, scale=True, tol=0.0001)
    
    # 训练模型
    svm_model.fit(X_train_whole,y_train_whole)
    lr_model.fit(X_train_whole, y_train_whole)
    pls_model.fit(X_train_whole, y_train_whole)

    # 预测
    svm_proba = svm_model.predict_proba(X_ind_test) 
    svm_proba = svm_proba[:, 1]

    lr_proba = lr_model.predict_proba(X_ind_test) 
    lr_proba = lr_proba[:, 1]

    pls_proba = pls_model.predict(X_ind_test) 
    pls_proba = np.clip(pls_proba, 0, 1)
    pls_proba = np.reshape(pls_proba, -1)   # 将二维数组展平为一维

    # 平均投票
    y_pred_proba = (lr_proba  + svm_proba + pls_proba + et_proba + nb_proba +knn_proba)/6
    y_valid_pred = np.where(y_pred_proba < 0.5, 0, 1)

    TP, FP, FN, TN = confusion_matrix(y_ind_test, y_valid_pred).ravel()
    Sn_collection.append(TP / (TP + FN))
    Sp_collection.append(TN / (TN + FP))
    MCC = (TP*TN-FP*FN)/np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
    MCC_collection.append(MCC)
    ACC_collection.append((TP + TN) / (TP + TN + FP + FN))
    BACC_collection.append(0.5 * (TP / (TP + FN)) + 0.5 * (TN / (TN + FP)))
    auc = roc_auc_score(y_ind_test, y_pred_proba)
    AUC_collection.append(auc)

print(round(statistics.mean(ACC_collection),3),'±',round(statistics.stdev(ACC_collection),3))
print(round(statistics.mean(Sn_collection),3),'±',round(statistics.stdev(Sn_collection),3))
print(round(statistics.mean(Sp_collection),3),'±',round(statistics.stdev(Sp_collection),3))
print(round(statistics.mean(MCC_collection),3),'±',round(statistics.stdev(MCC_collection),3))
print(round(statistics.mean(AUC_collection),3),'±',round(statistics.stdev(AUC_collection),3))
